In [1]:
import pandas as pd
import fileinput

In [2]:
inp = fileinput.FileInput('input/kinases_human.txt')
pkin = []
for line in inp:
    pkin.append(line.split('\n')[0])

In [3]:
df = pd.read_csv('input/reactome.homo_sapiens.interactions.tab-delimited.txt', delimiter = '\t')
#df = pd.read_csv('input/testinp.txt', delimiter = '\t')
prot1 = [i.split(':')[1] for i in list(df.iloc[:, 0])]
prot2 = [i.split(':')[1] for i in list(df.iloc[:, 3])]

In [4]:
print(prot1[1:10], prot2[1:10])

['P37840', 'P0DJI8', 'P06727', 'P01160', 'P51572', 'O95999', 'P05067', 'P05067', 'P05067'] ['P37840', 'P0DJI8', 'P06727', 'P01160', 'P51572', 'O95999', 'P05067', 'P05067', 'P05067']


In [5]:
### CONVERT FROM UNIPROT ID TO GENE ID

# UNIPROT ID TO GENE ID
df1 = pd.read_csv('input/uniprot_to_gene_hgnc.txt', sep = '\t')
df1 = df1.dropna()
df1.head()

,Approved symbol,UniProt ID(supplied by UniProt)
0,A1BG,P04217
2,A1CF,Q9NQ94
3,A2M,P01023
5,A2ML1,A8K2U0
9,A3GALT2,U3KPV4


In [6]:
# MAPS UNIPROT ID TO GENE ID
toGeneID = {} # maps uniprotID to approved symbol(s)
for x, y in zip(list(df1.loc[:, 'Approved symbol']), list(df1.loc[:, 'UniProt ID(supplied by UniProt)'])):
    if y not in toGeneID:
        toGeneID[y] = {x}
    else:
        toGeneID[y].add(x)

In [7]:
# find KSIs
d = {}
for p1, p2 in zip(prot1, prot2):
    if p1 in toGeneID and p2 in toGeneID:
        p1 = toGeneID[p1]
        p2 = toGeneID[p2]
        for p1x in p1:
            if p1x in pkin:
                if p1x not in d:
                    d[p1x] = {p2x for p2x in p2}
                else:
                    {d[p1x].add(p2x) for p2x in p2}
        for p2x in p2:
            if p2x in pkin:
                if p2x not in d:
                    d[p2x] = {p1x for p1x in p1}
                else:
                    {d[p2x].add(p1x) for p1x in p1}

In [8]:
allnp = open('output_reactome_all_human.gmt', 'w+')
fnp = open('output_reactome_fourplusinteractions_human.gmt', 'w+')
uSubs = set()
numKSI = 0
numKins = 0
for k in d:
    allnp.write('{0}_reactome_human\t'.format(k) + '\t'.join(d[k]) + '\n')
    if len(d[k]) >= 4:
        numKSI += len(d[k])
        numKins += 1
        {uSubs.add(x) for x in d[k]}
        fnp.write('{0}_reactome_human\t'.format(k) + '\t'.join(d[k]) + '\n')
    
        
print('{0}\t#kins: {1}\t#ksi: {2}\t#usubs: {3}'.format('human', numKins, numKSI, len(uSubs)))

allnp.close()
fnp.close()



human	#kins: 108	#ksi: 2659	#usubs: 1042
